# <p style="text-align: center;">IS590 Programmming for Analytics - Summer - Final Project </p>

### <p style="text-align: center;">Land Use Analysis Leveraging Numpy & Point Cloud Generated Scans</p>
**<p style="text-align: center;">Author - Jeremy Carnahan</p>**



## Background on Land Use Analytics

Aerial imagery has been used for many years, [predating even the airplane](https://en.wikipedia.org/wiki/Pigeon_photography), as people have used aerial perspectives for many industries.  One usecase in particular is for quickly accounting for how land is partitioned into uses such as farming, forestry, urbanization, roadways, etc. Flood planning utilize this distribution of land use to determine rainfall absorption and runoff for hydrology models.

For almost a century, measurement of land use has utilized a process called photogrammetry to extract surface areas of land use categories by measuring aerial photos using trigonometry.  This process is still quite effective, but is dependent on knowing the precise altitude above ground and camera intrinsics.  In the past few decades, a new technology called LiDAR (Light Detection And Ranging) leverages numerous laser pulses to scan terrain from the air, and return very accurate representation of features on the ground.  The data this produces is called a **pointcloud**, which is a 3-dimensional array of points in a Cartesian grid.  Further, this pointcloud undergoes a classification process using a standard called [American Society for Photogrammetry & Remote Sensing v1.4](http://www.asprs.org/wp-content/uploads/2019/03/LAS_1_4_r14.pdf).  This standard is used to delineate land use categories below, either through manual categorization or machine learning, to about 20 categories.  In the flood planning use case, typically the three categories used for hydrology modeling are: vegetation, road surfaces, and ground.  
 

## Hypothesis
_As pointclouds are classified 3-dimensional XYZ points using ASCII format, and are generally several gigabytes to terrabytes in size.  My hypothesis is that we can use Numpy to analyze these points to more quickly determine land use surface area, as Numpy is designed for n-dimension vectorization, and efficiently leverages memory for quick analysis.  If this approach is successful, it could substitute the numerous hours of manual measurement of aerial photography to determine vegegation and road surface area for flood modeling._

## Python Code
###### Import Modules
Here we need to pull in the modules we'll use to extract the pointcloud (.las file) into a proper Numpy ndArray using 'laspy'. 
From 'scipy' we'll use functions to calculate the area of classified points. 
'ipyleaflet' is used to get a course understanding of the area of interest by embedding a map we can calculate total area from.
'skimage.measure' is a module we can use to decimate our pointcloud for better visualization performance.

In [1]:
import laspy
from laspy import header
import scipy
import numpy as np
# ipyleaflet can be found by using 'conda install -c conda-forge ipyleaflet' in Anaconda command prompt
import ipyleaflet
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import path
from skimage.measure import block_reduce

###### Area of Interest and Map Extent
In order to download the LiDAR data from the [National Oceanic & Atmospheric Administration](https://coast.noaa.gov/dataviewer/#/lidar/search/), we need to determine the extent (bounding box) of the geography of interest.  ipyleaflet, which uses an open source tile mapping service using a product called Leaflet, can be embedded in Jupyter with the capability to determine overall surface area for the area of interest.  

In [2]:
# Using Leaflet opensource mapping tile service.  Used ipyleaflet docs here:
# https://ipyleaflet.readthedocs.io/en/latest/api_reference/map.html#usage 
# Added a polygon around the area of interest with ipyleaflet docs here:
# https://ipyleaflet.readthedocs.io/en/latest/api_reference/rectangle.html?highlight=draw%20polygon 
# Added an in-frame measurement tool from ipyleaflet docs here:
# https://ipyleaflet.readthedocs.io/en/latest/api_reference/measure_control.html?highlight=calculate%20area#methods
m = ipyleaflet.Map(center=(29.608362, -95.158648), zoom=13)


rectangle = ipyleaflet.Rectangle(bounds=((29.59172203, -95.17940998), (29.62288498, -95.13817692)))
m.add_layer(rectangle)

measure = ipyleaflet.MeasureControl(
    position='bottomleft',
    active_color = 'orange',
    primary_length_unit = 'kilometers',
    primary_area_unit = 'sqmeters'
)
m.add_control(measure)

measure.completed_color = 'blue'

measure.add_area_unit('sqmeters', 1, 4)
measure.secondary_area_unit = 'sqmiles'

m



Map(center=[29.608362, -95.158648], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

###### Importing the Pointcloud .las file
Now with our area of interest defined, and LiDAR data downloaded from NOAA, we can import the pointcloud using laspy to convert the .las ASCII file into a Numpy 3D array. 

Note: We'll load a significantly trimmed down .las pointcloud (~13Mb) from 3GB to allow for storage in Github.  However, as noted above, Numpy was chose because of its efficiency in computing large files.

We'll also do a little data exploration to determine the ASPRS standard version used, header, and point metadata.

In [46]:
def get_laspy_infile(directory: str) -> laspy.file.File:
    infile = laspy.file.File(directory, mode='r')
    return infile

In [50]:
# Read in the .las file to laspy file format and check details
# Based on laspy.py 'getting started' docs found here:
# https://pythonhosted.org/laspy/tut_part_1.html

def get_las_meta(inFile: laspy.file.File) -> list:
    
    metadata = []
    
    # We need to know what version of .las we're using so that we associate the right classification numbers
    # https://pythonhosted.org/laspy/_modules/laspy/header.html
    major = header.HeaderManager.get_majorversion(inFile)
    minor = header.HeaderManager.get_minorversion(inFile)
    las_version = str(major) + '.' + str(minor)
    metadata.append(las_version)

    # Here we look at the header and point formats to know what features are available
    # Code examples used from laspy.py doc page here:
    # https://pythonhosted.org/laspy/tut_part_1.html 

    # Header Fields
    header_data = []
    headerformat = inFile.header.header_format
    for spec in headerformat:
        header_data.append(spec.name)

    # Metadata explaination from Laspy documentation here:
    # https://pythonhosted.org//laspy/tut_background.html?highlight=classification
    # Point Metadata
    point_data = []
    pointformat = inFile.point_format
    for spec in inFile.point_format:
        point_data.append(spec.name)
    
    metadata.append(header_data)
    metadata.append(point_data)
    
    return metadata

First, let's get our LAS to Laspy file object back and set as a global variable so we can look at its metadata in more detail, and call it for future analysis.

In [48]:
laspy_infile = get_laspy_infile('./Data/ellington-segment.las')

In [51]:
las_metadata = get_las_meta(laspy_infile)
print('LAS Version: {}'.format(las_metadata[0]))
print()
print('Header Data: {}'.format(las_metadata[1]))
print()
print('Point Metadata: {}'.format(las_metadata[2]))

LAS Version: 1.4

Header Data: ['file_sig', 'file_source_id', 'global_encoding', 'proj_id_1', 'proj_id_2', 'proj_id_3', 'proj_id_4', 'version_major', 'version_minor', 'system_id', 'software_id', 'created_day', 'created_year', 'header_size', 'data_offset', 'num_variable_len_recs', 'data_format_id', 'data_record_length', 'legacy_point_records_count', 'legacy_point_return_count', 'x_scale', 'y_scale', 'z_scale', 'x_offset', 'y_offset', 'z_offset', 'x_max', 'x_min', 'y_max', 'y_min', 'z_max', 'z_min', 'start_wavefm_data_rec', 'start_first_evlr', 'num_evlrs', 'point_records_count', 'point_return_count']

Point Metadata: ['X', 'Y', 'Z', 'intensity', 'flag_byte', 'classification_flags', 'classification_byte', 'user_data', 'scan_angle', 'pt_src_id', 'gps_time', 'red', 'green', 'blue']


###### Data Size & Shape
Now that we've seen the metadata, we'll start to transform the raw Numpy extract from the .las in a more usable format.  Here we'll also take a look at the data's shape and size.

laspy.py documentation recommends extracting out each dimension from the .las file first, stacking, and transposing.  This is due to the way that the LiDAR systems organze the XYZ points in an orientation that differs from Numpy's XYZ/3D array. 

Recommendation found on laspy.py getting started page here: https://pythonhosted.org/laspy/tut_part_1.html

In [61]:
def transpose_las(inFile: laspy.file.File) -> np.array:
    # If you are interested in seeing how the data looks when not transposed, you can uncomment the block below.  Note that this 
    # will double the memory used for this project around 3GB.
    # original_data = np.array([inFile.x, inFile.y, inFile.z])
    # print(original_data.shape)
    # print('original_data type is: ' + str(type(original_data)))

    transposed_las = np.vstack([inFile.x, inFile.y, inFile.z]).transpose()
    return transposed_las

transposed_data = transpose_las(laspy_infile)
transposed_data_shape = transposed_data.shape
transposed_data_size = transposed_data.size
print('transposed_data type is: ' + str(type(transposed_data)))
print('transposed_data shape is: ' + str(transposed_data_shape))
print('transposed_data size is: ' + str(transposed_data_size))
print()
print()
print('Quick look with a head of 10 of XYZ points')
transposed_data[:10]

transposed_data type is: <class 'numpy.ndarray'>
transposed_data shape is: (378653, 3)
transposed_data size is: 1135959


Quick look with a head of 10 of XYZ points


array([[1.99823420e+05, 3.19961806e+06, 8.64000034e+00],
       [1.99823951e+05, 3.19961821e+06, 8.67000007e+00],
       [1.99823631e+05, 3.19961857e+06, 8.64000034e+00],
       [1.99823342e+05, 3.19961892e+06, 8.64000034e+00],
       [1.99824420e+05, 3.19961828e+06, 8.64000034e+00],
       [1.99824131e+05, 3.19961860e+06, 8.64000034e+00],
       [1.99823849e+05, 3.19961896e+06, 8.64000034e+00],
       [1.99823552e+05, 3.19961932e+06, 8.64000034e+00],
       [1.99824990e+05, 3.19961832e+06, 8.64000034e+00],
       [1.99824709e+05, 3.19961867e+06, 8.64000034e+00]])

###### Data Transformation
As noted in the background section above, we are only interested in vegetation and road surfaces when determining land use for precipitation absorption and runoff (note that other hydrology methods to determine water flow can be derived from a pointcloud, but is beyond the scope of this analysis).  

Our first step is to extract just the classification we are interested in, and we'll save them out to files and load the points back in, which helps us manage our memory more effectively. 

In [65]:
def extract_features(inFile: laspy.file.File, features: list):
    # Used a variation of this code to identify and aggregate LiDAR classifications
    # https://gis.stackexchange.com/questions/255833/classifying-lidar-ground-points-using-laspy
    # Categories takent from ASPRS Standard 1.4 http://www.asprs.org/wp-content/uploads/2019/03/LAS_1_4_r14.pdf    
    
    for feature in features:
        if feature == 'ground':                 
            land_use = np.where(inFile.Classification == 2)  # ground (2)
        elif feature == 'vegetation':
            # All vegetation is low (3), medium (4), and high classified vegetation (5) 
            land_use = np.where(np.logical_or(inFile.Classification == 3, inFile.Classification == 4, inFile.Classification == 5))
        elif feature == 'buildings':            
            land_use = np.where(inFile.Classification == 6)  # buildings (6)
        elif feature == 'water':   
            water = np.where(inFile.Classification == 9)  # water (9)
        elif feature == 'rail':
            rail = np.where(inFile.Classification == 10)  # rail (10)
        elif feature == 'roads':
            land_use = np.where(inFile.Classification == 11)   # road surfaces (11)
        elif feature == 'wires':
            land_use = np.where(np.logical_or(inFile.Classification == 13, inFile.Classification == 14)) # wires (13, 14)
        elif feature == 'transmission_tower':
            transmission_tower = np.where(inFile.Classification == 15)  # transmission tower (15)
        elif feature == 'bridge':
            bridge = roads = np.where(inFile.Classification == 17)  # bridge (17)
        elif feature == 'overhead':
            # overhead equipment (19) (e.g. conveyors, mining equip., traffic lights)
            overhead = np.where(inFile.Classification == 19)
        elif feature == 'snow':
            snow = np.where(inFile.Classification == 21)  # snow (21)

            # Outfile write to .las found on Laspy github example here:
            # https://github.com/laspy/laspy
        outFile = laspy.file.File('./Data/'+ feature + '.las', mode='w', header=inFile.header)
        outFile.points = inFile.points[land_use]
        outFile.close()
    
    return True

extract_features(laspy_infile, ['ground', 'vegetation', 'roads'])


True

###### Analyze New Arrays
Just a quick check that we've extracted the classifications that we wanted.

In [ ]:
# Here we read back in just the elements we want to calculate the surface area for
inFile_veg= laspy.file.File('./vegetation.las', mode='r')
vegetation_np = np.vstack([inFile_veg.x, inFile_veg.y, inFile_veg.z]).transpose()
veg_shape = vegetation_np.shape
print('vegetation_np shape is: ' + str(veg_shape))
veg_percent = veg_shape[0]/transposed_data_shape[0]
print('percentage of total points = ' + '{:.2%}'.format(veg_percent))
print()

inFile_roads= laspy.file.File('./roads.las', mode='r')
roads_np = np.vstack([inFile_roads.x, inFile_roads.y, inFile_roads.z]).transpose()
roads_shape = roads_np.shape
print('roads_np shape is: ' + str(roads_shape))
roads_percent = roads_shape[0]/transposed_data_shape[0]
print('percentage of total points = ' + '{:.2%}'.format(roads_percent))
print()

inFile_ground= laspy.file.File('./ground.las', mode='r')
ground_np = np.vstack([inFile_ground.x, inFile_ground.y, inFile_ground.z]).transpose()
ground_shape = ground_np.shape
print('ground_np shape is: ' + str(ground_shape))
ground_percent = ground_shape[0]/transposed_data_shape[0]
print('percentage of total points = ' + '{:.2%}'.format(ground_percent))

###### Data Visualization
Here we start to run into some issues.  While so far the analysis of the pointclouds in numpy ndarrays has been smooth, with ample memory, as soon as we begin to try to visualize the data in Jupyter the system begins to stagger.  Additionally, the limited screen space limits any detailed view we may want to have to examine the pointcloud.

Below are some attempts to visualize the data, but ultimately I chose to use an external tool called 'CloudCompare' to view the results.


In [ ]:

y = roads_np[:,0]
x = roads_np[:,1]
z = roads_np[:,2]


fig = plt.figure(figsize=[500, 300])
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, z, marker='.')
plt.show()




###### Pointcloud Decimation (Reduction)
Numpy arrays make it fairly easy to perform a reduction in the granularity of the pointcloud.  While we wouldn't want to do this if we want to maintain accuracy/precision, it may be a useful step for reducing the size for visualization.  Below I've used a skimage.measure method called 'block_reduce' that systematically reduces the size (and shape if desired) using an average of points every 100 points.

In [ ]:
# Used a function from scikit-image docs to decimate the pointcloud for better visualization
# https://scikit-image.org/docs/dev/api/skimage.measure.html#skimage.measure.block_reduce

reduce = block_reduce(roads_np, block_size=(100, 1), func=np.mean, cval=np.mean(roads_np))
print(reduce.shape)
print(type(reduce))

y = roads_np[:,0]
x = roads_np[:,1]
z = roads_np[:,2]


fig = plt.figure(figsize=[500, 300])
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, z, marker='.')
plt.show()


###### Land Use Surface Area (and some bad news...)
Now that we've extracted out just the land use points/elements that we want, we now need to perform our area calculations.  'scipy.spatial' has a method called ConvexHull, which we've seen previously in IS-590; however, this is a little different.  Even though the method is called ConvexHull, when given a 3D numpy array, the method actually peforms what is called a Delaunay triangulation invoked from Qhull class parent.  Effectively it connects the points with edges to create small triangle facets, and then calculates the area of each small triangle.  This has the added benefit over a simple aerial overlay in that Delaunay will also include additional area as it considers the z-axis in addition to just the x & y axis.  But there is bad news...

Unfortunately, it seems the scipy.spatial.ConvexHull method is unable to determine when it should not connect points that are not near to it.  A function that is used in pointcloud utilities (but not present in scipy) is to restrict the Delaunay to a preset threshold of "nearest neighbors", which results in **MANY** large facet areas to be calculated and inflating our area results.  As we see in the ConvexHull results below, the area calculated is many times larger than our starting surface area.

In [ ]:
# We use a ConvexHull method from scipy.spatial which is actually invoking a Delaunay triangulation process to create facets
# and calculate total area.
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.ConvexHull.html 
veg_delaunay = scipy.spatial.ConvexHull(vegetation_np)
roads_delaunay = scipy.spatial.ConvexHull(roads_np)
ground_delaunay = scipy.spatial.ConvexHull(ground_np)


print('Vegetation Area = ' + '{:,}'.format(veg_delaunay.area))
print('Road Surface Area = ' + '{:,}'.format(roads_delaunay.area))
print('Ground Area = ' + '{:,}'.format(ground_delaunay.area))
print()


## Learnings




## References:
[Pigeon Photography](https://en.wikipedia.org/wiki/Pigeon_photography)

[NOAA Data Access Viewer](https://coast.noaa.gov/dataviewer/#/lidar/search/)

[ASPRS LAS Specification 1.4 - R14](http://www.asprs.org/wp-content/uploads/2019/03/LAS_1_4_r14.pdf)

[Leaflet Opensource Tile Mapping for Python](https://ipyleaflet.readthedocs.io/en/latest/api_reference/map.html#usage)

[Laspy .las Editor for Python](https://pythonhosted.org/laspy/tut_part_1.html)

[Pointcloud Classification Parsing](https://gis.stackexchange.com/questions/255833/classifying-lidar-ground-points-using-laspy)

[block_reduce from scikit-learn](https://scikit-image.org/docs/dev/api/skimage.measure.html#skimage.measure.block_reduce)

[ConbexHull/Delaunay from scipy.org docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.ConvexHull.html)

[Github Markdown Cheatsheet](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet)
